# Serial Detection  

In diesem Notebook wird der Code besprochen, der nötig ist, damit Bilder in einem Ordner von einem untrainierten Neuronalen Netz klassifiziert werden können.

In [1]:
import torch, torchvision
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
from os.path import isfile, isdir
import os, json, cv2, random, time

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [8]:
path = "/home/julius/PowerFolders/Masterarbeit/1_Datensaetze/personData200/"

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")


In [10]:
def get_images(base_path, all_image_names):
    file_name_list = os.listdir(base_path)
    
    for element in file_name_list:
        if isfile(base_path + element) & (element.lower().endswith(".jpg")):
            all_image_names.add((base_path + element).replace(base_path, ''))
        elif isdir(base_path + element):
            get_images(base_path + element + "/", all_image_names)

    all_image_names = list(all_image_names)
    all_image_names.sort()
    return all_image_names

In [11]:
image_list = set()
image_list = get_images(path, image_list)
print(len(image_list))

200


In [12]:
def serial_Predictor(config, path, save_images = False):
    output_list = []
    output_path = "/".join(path.split("/")[:-3]) + "/detections/{}/{}/".format(path.split("/")[-2], time.strftime("%d,%m,%Y-%H,%M"))
    if not isdir(output_path):
        os.makedirs(output_path)

    for count, element in enumerate(image_list):
        picture = cv2.imread(path + element)
        predictor = DefaultPredictor(cfg)
        outputs = predictor(picture)

        predicted_classes = list(np.asarray(outputs["instances"].pred_classes.to("cpu")))
        prediction_scores =  list(np.asarray(outputs["instances"].scores.to("cpu")))
        prediction_scores = [float(element) for element in prediction_scores]
        thing_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
        predicted_classes_names = [thing_classes[element] for element in predicted_classes]
        output_dict = {}
        output_dict[element] = {"category_names": predicted_classes_names, "prediction_scores": prediction_scores}
        output_list.append(output_dict)

        print("[INFO] Predicted {}\t{}/{}".format(element, count+1, len(image_list)))

        if save_images:
            v = Visualizer(picture[:,:, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

            cv2.imwrite(output_path + element[:-4] + "_with_detections.jpg", out.get_image()[:, :, ::-1])
    with open(output_path + "detections.json", "w+") as output_file:
        json.dump(output_list, output_file)

In [13]:
serial_Predictor(cfg, path)

/home/julius/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
[INFO] Predicted eins1_(125).JPG	1/200
[INFO] Predicted eins1_(132).JPG	2/200
[INFO] Predicted eins1_(133).JPG	3/200
[INFO] Predicted eins1_(134).JPG	4/200
[INFO] Predicted eins1_(137).JPG	5/200
[INFO] Predicted eins1_(138).JPG	6/200
[INFO] Predicted eins1_(142).JPG	7/200
[INFO] Predicted eins1_(145).JPG	8/200
[INFO] Predicted eins1_(154).JPG	9/200
[INFO] Predicted eins1_(155).JPG	10/200
[INFO] Predicted eins1_(156).JPG	11/200
[INFO] Predicted eins1_(161).JPG	12/200
[INFO] Predicted eins1_(165).JPG	13/200
[INFO] Predicted eins1_(166).JPG	14/200
[INFO] Predicted eins1_(173).JPG	15/200
[INFO] Predicted eins1_(178).JPG	16/200
[INFO] Predicted eins1_(181).JPG